In [1]:
import os
import re
import numpy as np
from nltk import sent_tokenize
from sklearn.metrics import classification_report
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [2]:
with open('woolf.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

In [3]:
correct_sents = [re.sub('\n', ' ', elem) for elem in sent_tokenize(full_text)][750:800]
text = ' '.join(correct_sents)
correct_sents[39:40] = re.split('(?<=\.{3})\s', correct_sents[39])
correct_sents[43:44] = re.split('(?<=\.{3})\s', correct_sents[43])

In [4]:
correct_sents

['They had quarrelled.',
 'Why the right way to open a tin of beef, with Shakespeare on board, under conditions of such splendour, should have turned them to sulky schoolboys, none can tell.',
 'Tinned beef is cold eating, though; and salt water spoils biscuits; and the waves tumble and lollop much the same hour after hour--tumble and lollop all across the horizon.',
 'Now a spray of seaweed floats past-now a log of wood.',
 'Ships have been wrecked here.',
 'One or two go past, keeping their own side of the road.',
 'Timmy knew where they were bound, what their cargoes were, and, by looking through his glass, could tell the name of the line, and even guess what dividends it paid its shareholders.',
 'Yet that was no reason for Jacob to turn sulky.',
 'The Scilly Isles had the look of mountain-tops almost a-wash...',
 'Unfortunately, Jacob broke the pin of the Primus stove.',
 'The Scilly Isles might well be obliterated by a roller sweeping straight across.',
 'But one must give young 

In [5]:
first_regex = '(?<=[.!?]) (?=[A-Z])'
first_split = [elem for elem in re.split(first_regex, text)]

In [6]:
second_regex = '(?<=[.!?])\s(?=[^a-z])'
second_split = [elem for elem in re.split(second_regex, text)]

In [7]:
print("Процент правильных ответов по метрике с множествами (first_regex): {}".format(len(set(correct_sents) & set(first_split)) / len(set(correct_sents) | set(first_split))))
print("Процент правильных ответов по метрике с множествами (second_regex): {}".format(len(set(correct_sents) & set(second_split)) / len(set(correct_sents) | set(second_split))))

Процент правильных ответов по метрике с множествами (first_regex): 0.9433962264150944
Процент правильных ответов по метрике с множествами (second_regex): 1.0


In [8]:
# Посмотрим с помощью операции симметрической разности, на каких предложениях ошибается первая регулярка
set(correct_sents) ^ set(first_split)

{'"Now..." said Jacob.',
 "Timmy Durrant's notebook of scientific observations.",
 'Timmy Durrant\'s notebook of scientific observations. "Now..." said Jacob.'}

#### Как видно, первая регулярка ошибается на {. "}, поскольку она воспринимает как начало следующего предложения только те случаи, когда предложение начинается с заглавной буквы. Вторая регулярка не страдает этим, поскольку для неё важно, чтобы следующее предложение начиналось не со строчной буквы, что верно и в этом случае.

### Можно также посчитать accuracy, precision, recall и f1-меру на униграммах и биграммах (униграммы не должны разбиваться, в отличие от биграмм). Сделаем это как вручную, так и с помощью встроенных функций

In [9]:
def find_table(regex):
    tp = 0
    fp = 0
    fn = 0
    tn = 0 # для accuracy необходимо значение и true negative

    for sent in correct_sents:
        if len(re.split(regex, sent)) == 1:
            tp += 1
        else:
            fp += 1

    for i in range(len(correct_sents) - 1):
        sent = ' '.join([correct_sents[i], correct_sents[i + 1]])
        if len(re.split(regex, sent)) == 2:
            tn += 1
        else:
            fn += 1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    print('tp: {}, fp: {}, fn: {}, tn: {}'.format(tp, fp, fn, tn))
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1: ', f1)
    print('Accuracy: ', accuracy)

In [10]:
find_table(first_regex)

tp: 52, fp: 0, fn: 1, tn: 50
Precision:  1.0
Recall:  0.9811320754716981
F1:  0.9904761904761905
Accuracy:  0.9902912621359223


In [11]:
find_table(second_regex)

tp: 52, fp: 0, fn: 0, tn: 51
Precision:  1.0
Recall:  1.0
F1:  1.0
Accuracy:  1.0


### Здесь видно, что первая регулярка не может разбить предложение 'Timmy Durrant's notebook of scientific observations. "Now..." said Jacob.' (fn = 1), однако fn не входит в формулу precision, поэтому precision = 1. Со второй регуляркой все метрики равны 1, поскольку ошибок нет.

In [12]:
first_borders = [0 for symbol in text]
cur_length = 0
for sent in first_split:
    cur_length += len(sent)
    first_borders[cur_length - 1] = 1
    cur_length += 1
second_borders = [0 for symbol in text]
cur_length = 0
for sent in second_split:
    cur_length += len(sent)
    second_borders[cur_length - 1] = 1
    cur_length += 1
correct_borders = [0 for symbol in text]
cur_length = 0
for sent in correct_sents:
    cur_length += len(sent)
    correct_borders[cur_length - 1] = 1
    cur_length += 1

In [13]:
np.all(first_borders == correct_borders), np.all(second_borders == correct_borders)

(False, True)

In [14]:
index = np.where(np.equal(first_borders, correct_borders) == False)[0][0]
first_borders[index], correct_borders[index]

(0, 1)

In [15]:
print(classification_report(correct_borders, first_borders))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      4899
          1       1.00      0.98      0.99        52

avg / total       1.00      1.00      1.00      4951



In [16]:
print(classification_report(correct_borders, second_borders))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      4899
          1       1.00      1.00      1.00        52

avg / total       1.00      1.00      1.00      4951



### Здесь мы рассмотрели задачу определения границ предложения как задачу классификации: в последовательности символов единицей закодированы те, что на границе предложений, а нулями -- все остальные. Precision в случае 0 first_borders не равна ровно 1, а скорее равна 4898/4899, но, видимо, здесь это значение округляется, поскольку оно очень близко к 1. Соответственно, это верно и для f1-score в случае 0.

In [17]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
"""
this includes as potential collocations all word pairs where the first
word ends in a period. It may be useful in corpora where there is a lot
of variation that makes abbreviations like Mr difficult to identify.
"""
trainer.train((' '.join(full_text.split())).replace(text, '')) # удалили участок, на котором будем тестировать

tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [18]:
tokenizer.PUNCTUATION, tokenizer._params.abbrev_types # аббревиатуры и знаки препинания

((';', ':', ',', '.', '!', '?'),
 {'1.a',
  '1.b',
  '1.d',
  '1.e.2',
  '1.e.3',
  '1.e.4',
  '1.e.5',
  '1.e.6',
  '1.e.9',
  '1.f',
  '1.f.1',
  '1.f.2',
  '1.f.4',
  '1.f.5',
  '1.f.6',
  'a.b.c',
  'dr',
  'e.m',
  'esq',
  'etc',
  'f3',
  'mr',
  'mrs',
  'r.b',
  'rev',
  's',
  'st',
  'u.s'})

In [19]:
tokenizer_split = tokenizer.tokenize(text)

In [20]:
tp = 0
fp = 0
fn = 0
tn = 0 # для accuracy необходимо значение и true negative

for sent in correct_sents:
    if len(tokenizer.tokenize(sent)) == 1:
        tp += 1
    else:
        fp += 1

for i in range(len(correct_sents) - 1):
    sent = ' '.join([correct_sents[i], correct_sents[i + 1]])
    if len(tokenizer.tokenize(sent)) == 2:
        tn += 1
    else:
        fn += 1
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
print('tp: {}, fp: {}, fn: {}, tn: {}'.format(tp, fp, fn, tn))
print('Precision: ', precision)
print('Recall: ', recall)
print('F1: ', f1)
print('Accuracy: ', accuracy)

tp: 52, fp: 0, fn: 2, tn: 49
Precision:  1.0
Recall:  0.9629629629629629
F1:  0.9811320754716981
Accuracy:  0.9805825242718447


In [21]:
set(tokenizer_split) ^ set(correct_sents)

{"Let's dry ourselves, and take up the first thing that comes handy...",
 "Let's dry ourselves, and take up the first thing that comes handy... Timmy Durrant's notebook of scientific observations.",
 'No matter.',
 "Timmy Durrant's notebook of scientific observations.",
 'Timmy sometimes wondered (only for a second) whether his people bothered him...',
 'Timmy sometimes wondered (only for a second) whether his people bothered him... No matter.'}

### Встроенный tokenizer ведёт себя хуже, чем first_regex и тем более чем second_regex. Видимо, проблема в том, что tokenizer не воспринимает ... (а не …) как целый разделитель. Поэтому попробуем добавить этот разделитель в tokenizer.PUNCTUATION и проверим качество:

In [22]:
tokenizer.PUNCTUATION = tuple(list(tokenizer.PUNCTUATION) + ['...'])

In [23]:
tokenizer.PUNCTUATION

(';', ':', ',', '.', '!', '?', '...')

In [24]:
tokenizer_split = tokenizer.tokenize(text)

In [25]:
set(tokenizer_split) ^ set(correct_sents)

{"Let's dry ourselves, and take up the first thing that comes handy...",
 "Let's dry ourselves, and take up the first thing that comes handy... Timmy Durrant's notebook of scientific observations.",
 'No matter.',
 "Timmy Durrant's notebook of scientific observations.",
 'Timmy sometimes wondered (only for a second) whether his people bothered him...',
 'Timmy sometimes wondered (only for a second) whether his people bothered him... No matter.'}

### Не сработало. Попробуем изменить в тексте все троеточия на один символ троеточия

In [26]:
full_text = full_text.replace('...', '…')
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train((' '.join(full_text.split())).replace(text, ''))

tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [27]:
tokenizer.PUNCTUATION

(';', ':', ',', '.', '!', '?')

In [28]:
tokenizer.PUNCTUATION = tuple(list(tokenizer.PUNCTUATION) + ['…'])

In [29]:
tokenizer_split = tokenizer.tokenize(text.replace('...', '…'))

In [30]:
correct_sents = [elem.replace('...', '…') for elem in correct_sents]

In [31]:
set(correct_sents) ^ set(tokenizer_split)

{"Let's dry ourselves, and take up the first thing that comes handy…",
 "Let's dry ourselves, and take up the first thing that comes handy… Timmy Durrant's notebook of scientific observations.",
 'No matter.',
 'The Scilly Isles had the look of mountain-tops almost a-wash…',
 'The Scilly Isles had the look of mountain-tops almost a-wash… Unfortunately, Jacob broke the pin of the Primus stove.',
 "Timmy Durrant's notebook of scientific observations.",
 'Timmy sometimes wondered (only for a second) whether his people bothered him…',
 'Timmy sometimes wondered (only for a second) whether his people bothered him… No matter.',
 'Unfortunately, Jacob broke the pin of the Primus stove.'}

### Но и это не помогает (стало ещё хуже), поскольку, по-видимому, проблема в не типографском знаке (его отсутствии), а в том, что PunktSentenceTokenizer рассматривает случаи выше как продолжение предыдущего, хотя, на мой взгляд, если бы это было действительно так, то продолжение писалось бы со строчной буквы (см. пример из BrE "It is not cold… it is freezing cold." из https://en.wikipedia.org/wiki/Ellipsis). Возможно, в случае "Timmy Durrant's notebook of scientific observations" tokenizer размечает и правильно (трудно понять, т.к. имя всегда пишется с заглавной буквы), но в двух других случаях скорее неправильно.

### Покажем, что некоторые ошибки из других отрывков мы можем исправить добавлением сокращений в tokenizer._params.abbrev_types

In [32]:
for i, sent in enumerate([re.sub('\n', ' ', elem) for elem in sent_tokenize(full_text)]):
    if sent.startswith('Having drawn her water'):
        print(i)

856


In [33]:
hard_sent = [re.sub('\n', ' ', elem) for elem in sent_tokenize(full_text)][856]
tokenizer.tokenize(hard_sent)

['Having drawn her water, Mrs. Pascoe went in.']

### Сокращение Mrs уже добавлено в tokenizer._params.abbrev_types, поэтому не сработало. Удалим из этого множества этот элемент и убедимся в том, что теоретически в каком-нибудь другом случае добавление аббревиатуры может помочь

In [34]:
tokenizer._params.abbrev_types -= {'mrs'}

In [35]:
tokenizer._params.abbrev_types

{'1.a',
 '1.b',
 '1.d',
 '1.e.2',
 '1.e.3',
 '1.e.4',
 '1.e.5',
 '1.e.6',
 '1.e.9',
 '1.f',
 '1.f.1',
 '1.f.2',
 '1.f.4',
 '1.f.5',
 '1.f.6',
 'a.b.c',
 'dr',
 'e.m',
 'esq',
 'etc',
 'f3',
 'mr',
 'r.b',
 'rev',
 's',
 'st',
 'u.s'}

In [36]:
tokenizer.tokenize(hard_sent)

['Having drawn her water, Mrs. Pascoe went in.']

### Думаю, проблема в том, что токенизатор уже выучил данное предложение. Попробуем обучить его без этого предложения.

In [37]:
full_text = full_text.replace('...', '…')
trainer = PunktTrainer()
trainer.train((' '.join(full_text.split())).replace(text, '').replace("Having drawn her water, Mrs. Pascoe went in.", ""))

tokenizer = PunktSentenceTokenizer(trainer.get_params())
tokenizer._params.abbrev_types -= {'mrs'}

In [38]:
tokenizer.tokenize(hard_sent)

['Having drawn her water, Mrs.', 'Pascoe went in.']

In [40]:
tokenizer._params.abbrev_types |= {'mrs'}
tokenizer.tokenize(hard_sent)

['Having drawn her water, Mrs. Pascoe went in.']